In [1]:
import json
import urllib.request
import cv2
import imageio
import numpy as np
import webcolors
from contextlib import closing
from urllib.request import urlopen
import os

In [6]:
#REMEMBER TO CHANGE THE DIRECTORIES TO MATCH YOURS
path = "C:/Users/Admin/Desktop/jun/University/Year 4/GDP/work/code/notebooks/run/ok/real images/masks.json" #exported json file's directory
mask_output_dir = "C:/Users/Admin/Desktop/jun/University/Year 4/GDP/work/code/notebooks/run/ok/real images/masks/" #directory to save downloaded masks
classes_dir = "C:/Users/Admin/Desktop/jun/University/Year 4/GDP/work/code/notebooks/run/ok/UNIMIB2016 Food Database/food_class.txt"


In [7]:
f = open(classes_dir, "r")
classes = [x.replace('\n', '').replace(' ','_') for x in f ]

k = 5 #if generating food masks
#k = 50 #if generating plate masks

values = [255-k*n for n in range(len(classes))] #color code for each class


In [8]:
values

[255, 250, 245, 240, 235, 230, 225, 220, 215, 210, 205, 200]

In [9]:
with open(path) as f: #load json file
    labelbox = json.load(f)

In [6]:
food_count = {}
food_by_files = {}

n=0
for file in labelbox:
    if len(file['Label']) !=0:
        objs = file['Label']['objects']
        for food in objs:
            food_name = food['value']
            if food_name not in food_count:
                food_count[food_name] = 1
            else:
                food_count[food_name] = food_count[food_name] + 1
                
            if food_name not in food_by_files:
                food_by_files[food_name] = [file['Labeled Data'][94:file['Labeled Data'].index('?')-4]]
            else:
                food_by_files[food_name].append(file['Labeled Data'][94:file['Labeled Data'].index('?')-4])

    
#labelbox[10]['Label']['objects'][0]['value']

In [7]:
def run_url(url): #this function tries to open the the url from json file
    while True:
        try:
            with closing(urllib.request.urlopen(url)) as su:
                img_read = su.read()  
            return img_read    
        except:
            pass

In [9]:
with open(path) as f: #load json file
    data = json.load(f)

failed = ['PHO_037_2019_04_05_A','PHO_056_2019_02_07_B','PHO_037_2019_04_04_B','PHO_033_2019_06_19_B'] #extract these from food2.json
    
n=0
for d in data:
    file_name = d['Labeled Data'][94:d['Labeled Data'].index('?')-4]

    #print(n,file_name)
    if file_name not in failed:
        if len(d['Label']) !=0:
            image = d['Labeled Data']
            objects = d['Label']['objects']

            first_seg = True
            for food in objects:
                food_name = food['value']
                food_url = food['instanceURI']

                mask = run_url(food_url)
                mask = np.asarray(bytearray(mask), dtype="uint8")
                mask = cv2.imdecode(mask, cv2.IMREAD_COLOR) 

                mask[mask>0] = 255 - (classes.index(food_name)+1)*5

                if first_seg:
                    mask_merge = mask
                    first_seg = False

                else:
                    mask_merge = mask_merge | mask

            #print(mask_merge.shape)
            mask_name = d['Labeled Data'][94:d['Labeled Data'].index('?')-4]+'.png'
            imageio.imwrite(mask_output_dir + mask_name ,mask_merge) #downloads the finalized masks
            print(n,mask_name,"saved")
            n=n+1

        else:
            image_url = d['Labeled Data']
            image = run_url(image_url)
            image = np.asarray(bytearray(image), dtype="uint8")
            image = cv2.imdecode(image, cv2.IMREAD_COLOR) #image matrix with 3 channels
            img_shape = image.shape
            blank_img = np.zeros(img_shape)
            blank_img = blank_img.astype('int64') 

            mask_name = d['Labeled Data'][94:d['Labeled Data'].index('?')-4]+'.png'
            imageio.imwrite(mask_output_dir + mask_name,blank_img) #downloads the finalized masks
            print(n,mask_name,"saved")
            n=n+1

print('finished')

0 PHO_012_2019_05_09_B.png saved
1 PHO_033_2019_06_19_B.png saved
2 PHO_047_2019_06_17_A_LI.png saved
3 PHO_071_2019_08_30_B.png saved
4 PHO_091_2019_02_14_B.png saved
5 PHO_013_2019_07_25_B.png saved
6 PHO_033_2019_06_21_A.png saved
7 PHO_055_2019_03_28_B.png saved
8 PHO_077_2019_05_02_A.png saved
9 PHO_095_2019_07_19_A_LI.png saved
10 PHO_014_2019_01_23_A_LI.png saved
11 PHO_037_2019_04_03_B.png saved
12 PHO_056_2019_02_06_A_LI.png saved
13 PHO_077_2019_05_03_B.png saved
14 PHO_095_2019_07_22_B.png saved
15 PHO_018_2019_03_01_B.png saved
16 PHO_040_2019_08_02_A.png saved
17 PHO_057_2019_03_28_B.png saved
18 PHO_080_2019_05_31_B.png saved
19 PHO_004_2019_08_09_A.png saved
20 PHO_019_2019_07_18_A_LI.png saved
21 PHO_042_2019_02_21_A_LI.png saved
22 PHO_059_2019_03_11_A.png saved
23 PHO_088_2019_05_29_A.png saved
24 PHO_006_2019_04_26_B_LI.png saved
25 PHO_014_2019_01_25_B.png saved
26 PHO_079_2019_03_23_A_LI.png saved
27 PHO_003_2019_01_30_A_LI.png saved
28 PHO_018_2019_02_28_A.png sav

In [15]:
#REMEMBER TO CHANGE THE DIRECTORIES TO MATCH YOURS
path = "C:/Users/Admin/Desktop/jun/University/Year 4/GDP/work/code/notebooks/run/dataset/hospital_data/food2.json" #exported json file's directory
mask_output_dir = "C:/Users/Admin/Desktop/jun/University/Year 4/GDP/work/code/notebooks/run/dataset/hospital_data/faulty/" #directory to save downloaded masks
classes_dir = "C:/Users/Admin/Desktop/jun/University/Year 4/GDP/work/code/notebooks/run/dataset/hospital_data/food_class.txt"


In [16]:
with open(path) as f: #load json file
    data = json.load(f)

failed = ['PHO_037_2019_04_05_A','PHO_056_2019_02_07_B','PHO_037_2019_04_04_B','PHO_033_2019_06_19_B']
    
n=0
for d in data:
    file_name = d['Labeled Data'][94:d['Labeled Data'].index('?')-4]

    #print(n,file_name)
    if file_name in failed:
        if len(d['Label']) !=0:
            image = d['Labeled Data']
            objects = d['Label']['objects']

            first_seg = True
            for food in objects:
                food_name = food['value']
                food_url = food['instanceURI']

                mask = run_url(food_url)
                mask = np.asarray(bytearray(mask), dtype="uint8")
                mask = cv2.imdecode(mask, cv2.IMREAD_COLOR) 

                mask[mask>0] = 255 - (classes.index(food_name)+1)*5

                if first_seg:
                    mask_merge = mask
                    first_seg = False

                else:
                    mask_merge = mask_merge | mask

            #print(mask_merge.shape)
            mask_name = d['Labeled Data'][94:d['Labeled Data'].index('?')-4]+'.png'
            imageio.imwrite(mask_output_dir + mask_name ,mask_merge) #downloads the finalized masks
            print(n,mask_name,"saved")
            n=n+1

        else:
            image_url = d['Labeled Data']
            image = run_url(image_url)
            image = np.asarray(bytearray(image), dtype="uint8")
            image = cv2.imdecode(image, cv2.IMREAD_COLOR) #image matrix with 3 channels
            img_shape = image.shape
            blank_img = np.zeros(img_shape)
            blank_img = blank_img.astype('int64') 

            mask_name = d['Labeled Data'][94:d['Labeled Data'].index('?')-4]+'.png'
            imageio.imwrite(mask_output_dir + mask_name,blank_img) #downloads the finalized masks
            print(n,mask_name,"saved")
            n=n+1

print('finished')

0 PHO_037_2019_04_05_A.png saved
1 PHO_033_2019_06_19_B.png saved
2 PHO_037_2019_04_04_B.png saved
finished
